In [16]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna
import os
import json 


conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
    SELECT DISTINCT
        f.match_date,
        f.team,
        f.is_home,
        f.gw,
        f.[is_promoted?],
        f.[is_early_season?],
        f.season,
        team_elo.elo as team_elo,
        f.team_rolling_summary_goals,
        f.team_rolling_summary_pens_made,
        f.team_rolling_summary_shots,
        f.team_rolling_summary_shots_on_target,
        f.team_rolling_summary_touches,
        f.team_rolling_summary_xg,
        f.team_rolling_summary_npxg,
        f.team_rolling_possession_touches_att_pen_area,
        f.team_rolling_keeper_psxg,
        f.team_rolling_conceded_summary_goals,
        f.team_rolling_conceded_summary_pens_made,
        f.team_rolling_conceded_summary_shots,
        f.team_rolling_conceded_summary_shots_on_target,
        f.team_rolling_conceded_summary_touches,
        f.team_rolling_conceded_summary_xg,
        f.team_rolling_conceded_summary_npxg,
        f.team_rolling_conceded_possession_touches_att_pen_area,
        f.team_rolling_conceded_keeper_psxg,
        f.opp_team,
        opp_elo.elo as opp_team_elo,
        f.opp_team_rolling_summary_goals,
        f.opp_team_rolling_summary_pens_made,
        f.opp_team_rolling_summary_shots,
        f.opp_team_rolling_summary_shots_on_target,
        f.opp_team_rolling_summary_touches,
        f.opp_team_rolling_summary_xg,
        f.opp_team_rolling_summary_npxg,
        f.opp_team_rolling_possession_touches_att_pen_area,
        f.opp_team_rolling_keeper_psxg,
        f.opp_team_rolling_conceded_summary_goals,
        f.opp_team_rolling_conceded_summary_pens_made,
        f.opp_team_rolling_conceded_summary_shots,
        f.opp_team_rolling_conceded_summary_shots_on_target,
        f.opp_team_rolling_conceded_summary_touches,
        f.opp_team_rolling_conceded_summary_xg,
        f.opp_team_rolling_conceded_summary_npxg,
        f.opp_team_rolling_conceded_possession_touches_att_pen_area,
        f.opp_team_rolling_conceded_keeper_psxg,
        ms.summary_goals as goals
    FROM 
        team_all_features_180_0077 f
    JOIN
        fbref_match_all_columns ms
            ON ms.match_url = f.match_url
            AND ms.team = f.team
    JOIN
        clubelo_features team_elo
            ON team_elo.fbref_team = f.team
                AND DATE(f.match_date) BETWEEN team_elo.start_date AND team_elo.end_date
    JOIN
        clubelo_features opp_elo
            ON opp_elo.fbref_team = f.opp_team
                AND DATE(f.match_date) BETWEEN opp_elo.start_date AND opp_elo.end_date
    WHERE 
        f.team_rolling_summary_minutes IS NOT NULL
        AND f.division = 'Premier League'
                       """, conn)

df

,match_date,team,is_home,gw,is_promoted?,is_early_season?,season,team_elo,team_rolling_summary_goals,team_rolling_summary_pens_made,...,opp_team_rolling_conceded_summary_goals,opp_team_rolling_conceded_summary_pens_made,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,goals
0,2025-05-25 00:00:00,Tottenham,1,38,0,0,2024-2025,1778.323242,1.289346,0.099220,...,1.604183,0.239586,10.857144,3.427652,593.691838,1.339005,1.150561,23.135899,1.269035,1.0
1,2025-05-25 00:00:00,Brighton,0,38,0,0,2024-2025,1809.790894,1.756276,0.301971,...,2.000341,0.043949,14.976592,5.211294,577.666107,1.803889,1.770927,27.321844,1.890708,4.0
2,2025-05-25 00:00:00,Bournemouth,1,38,0,0,2024-2025,1806.686401,1.620226,0.133736,...,2.002831,0.111713,16.524494,4.915440,671.768114,1.915188,1.787126,29.916176,1.738255,2.0
3,2025-05-25 00:00:00,Leicester City,0,38,1,0,2024-2025,1620.328247,0.767395,0.000000,...,1.341725,0.163690,12.755894,4.204451,593.777537,1.319921,1.188969,24.624059,1.354963,0.0
4,2025-05-25 00:00:00,Newcastle Utd,1,38,0,0,2024-2025,1878.094604,2.071963,0.170780,...,1.053137,0.000000,12.095816,4.350498,669.491722,1.138803,1.112165,23.319336,1.247472,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6016,2020-09-12 00:00:00,Leeds United,0,1,1,1,2020-2021,1624.624268,2.334007,0.120432,...,1.375240,0.105527,7.383654,2.586850,493.308949,1.196898,1.112476,10.199228,1.124304,3.0
6017,2020-09-12 00:00:00,Crystal Palace,1,1,0,1,2020-2021,1673.688843,0.588576,0.000000,...,0.806965,0.000000,10.857775,2.994314,554.864046,1.218138,1.218138,20.902557,1.221304,1.0
6018,2020-09-12 00:00:00,Southampton,0,1,0,1,2020-2021,1720.211182,1.922152,0.136975,...,1.992121,0.000000,12.932078,4.419749,690.958959,1.499584,1.499584,21.356376,1.824323,0.0
6019,2020-09-12 00:00:00,Fulham,1,1,1,1,2020-2021,1591.546265,1.309914,0.120436,...,1.172560,0.176528,13.505428,4.917995,576.533198,1.614600,1.476587,23.536873,1.376875,0.0


In [17]:
X = df.drop(columns=["team", "opp_team", "goals", "match_date"])
X = pd.get_dummies(X, columns=["season"], drop_first=True)

#X = df[["is_home", "team_rolling_summary_goals", "opp_team_rolling_conceded_summary_goals"]] # goals only

y = df["goals"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'
df['match_date'] = pd.to_datetime(df['match_date'])

train_mask = df['match_date'] < cutoff_date
test_mask = df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

# Create DMatrix objects BEFORE using them in eval_list
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'count:poisson',
    'max_depth': 7,
    'eta': 0.1,
    'subsample': 0.7,
    'colsample_bytree': 0.8,
    'eval_metric': 'poisson-nloglik'
}

num_rounds = 100
eval_list = [(dtrain, 'train'), (dtest, 'eval')]  # Now dtrain and dtest are defined

model = xgb.train(
    params, 
    dtrain, 
    num_rounds, 
    eval_list, 
    early_stopping_rounds=10,
    verbose_eval=10
)

# Make predictions
preds = model.predict(dtest)

rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

[0]	train-poisson-nloglik:1.55110	eval-poisson-nloglik:1.55038
[10]	train-poisson-nloglik:1.43518	eval-poisson-nloglik:1.49292
[20]	train-poisson-nloglik:1.37516	eval-poisson-nloglik:1.48180
[30]	train-poisson-nloglik:1.33167	eval-poisson-nloglik:1.47927


c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[40]	train-poisson-nloglik:1.30261	eval-poisson-nloglik:1.47754
[50]	train-poisson-nloglik:1.27490	eval-poisson-nloglik:1.47553
[59]	train-poisson-nloglik:1.25561	eval-poisson-nloglik:1.47572
RMSE: 1.1591
MAE: 0.9141


In [18]:
# Get feature importance
importance = model.get_score(importance_type='gain')
feature_names = X_train.columns.tolist()

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'importance': list(importance.values())
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features:")
print(importance_df.head(15))

Top 10 Most Important Features:
                                              feature  importance
4                                            team_elo    5.827780
23                                       opp_team_elo    5.120790
12       team_rolling_possession_touches_att_pen_area    3.570798
38               opp_team_rolling_conceded_summary_xg    3.534540
13                           team_rolling_keeper_psxg    3.291078
35            opp_team_rolling_conceded_summary_shots    3.245657
47                                   season_2023-2024    3.004133
40  opp_team_rolling_conceded_possession_touches_a...    2.933122
44                                   season_2020-2021    2.849222
36  opp_team_rolling_conceded_summary_shots_on_target    2.800586
43                                   season_2019-2020    2.767511
26                     opp_team_rolling_summary_shots    2.764109
10                            team_rolling_summary_xg    2.728385
17      team_rolling_conceded_summary_shots_

# Hyperparameter Tuning

In [23]:
# Set to True to tune new parameters, False to load existing
TUNE_NEW_PARAMS = True  # Change this to False to load saved params

# Tuning settings
N_TRIALS = 150
PARAMS_FILE = "best_xgb_params_prem_simple.json"

In [24]:
if TUNE_NEW_PARAMS or not os.path.exists(PARAMS_FILE):
    print("Tuning new hyperparameters...")
    
    def objective(trial):
        # Optimized search space for ~6k rows, ~50 features
        params = {
            'objective': 'count:poisson',
            'eval_metric': 'poisson-nloglik',
            'max_depth': trial.suggest_int('max_depth', 3, 8),  # Conservative for dataset size
            'eta': trial.suggest_float('eta', 0.01, 0.15, log=True),  # Balanced learning rate
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),  # Less aggressive sampling
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # Appropriate for 6k samples
            'gamma': trial.suggest_float('gamma', 0, 5),  # Minimum loss reduction
            'alpha': trial.suggest_float('alpha', 0, 10),  # L1 regularization
            'lambda': trial.suggest_float('lambda', 0.1, 10),  # L2 regularization - more important
            'max_leaves': trial.suggest_int('max_leaves', 0, 100),  # Moderate complexity
            'random_state': 42
        }
        
        n_estimators = trial.suggest_int('n_estimators', 100, 800)  # Extended range with early stopping
        
        # Train model
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=n_estimators,
            evals=[(dtrain, 'train'), (dtest, 'eval')],
            early_stopping_rounds=50,  # More patient early stopping
            verbose_eval=False
        )
        
        # Calculate RMSE
        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        
        return rmse
    
    # Run optimization with TPE sampler for better efficiency
    study = optuna.create_study(direction='minimize', 
                               sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)
    
    # Get best parameters
    best_params = study.best_params.copy()
    n_estimators = best_params.pop('n_estimators')
    
    print(f"Best RMSE: {study.best_value:.4f}")
    print("Best parameters:")
    for key, value in best_params.items():
        if key not in ['objective', 'eval_metric', 'random_state']:
            print(f"  {key}: {value}")
    print(f"  n_estimators: {n_estimators}")
    
    # Save parameters
    params_to_save = {
        'best_params': best_params,
        'n_estimators': n_estimators,
        'best_rmse': study.best_value
    }
    
    with open(PARAMS_FILE, 'w') as f:
        json.dump(params_to_save, f, indent=2)
    print(f"\nParameters saved to {PARAMS_FILE}")
    
else:
    print("Loading saved hyperparameters...")
    
    with open(PARAMS_FILE, 'r') as f:
        saved_params = json.load(f)
    
    best_params = saved_params['best_params']
    n_estimators = saved_params['n_estimators']
    
    print("Loaded parameters:")
    for key, value in best_params.items():
        if key not in ['objective', 'eval_metric', 'random_state']:
            print(f"  {key}: {value}")
    print(f"  n_estimators: {n_estimators}")
    print(f"Previous best RMSE: {saved_params['best_rmse']:.4f}")

[I 2025-08-05 09:48:32,871] A new study created in memory with name: no-name-e6da1fa8-f93a-44d6-8791-0876f442a81a


Tuning new hyperparameters...


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2025-08-05 09:48:33,135] Trial 0 finished with value: 1.1670403655464723 and parameters: {'max_depth': 5, 'eta': 0.13125830316209655, 'subsample': 0.892797576724562, 'colsample_bytree': 0.8394633936788146, 'colsample_bylevel': 0.6624074561769746, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'alpha': 8.661761457749352, 'lambda': 6.051038616257767, 'max_leaves': 71, 'n_estimators': 114}. Best is trial 0 with value: 1.1670403655464723.
[I 2025-08-05 09:48:33,493] Trial 1 finished with value: 1.1550188636908632 and parameters: {'max_depth': 8, 'eta': 0.09528587217040241, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.6727299868828402, 'colsample_bylevel': 0.6733618039413735, 'min_child_weight': 4, 'gamma': 2.6237821581611893, 'alpha': 4.319450186421157, 'lambda': 2.983168487960615, 'max_leaves': 61, 'n_estimators': 197}. Best is trial 1 with value: 1.1550188636908632.
[I 2025-08-05 09:48:33,856] Trial 2 finished with value: 1.161449770407236 and parameters: {'max_depth': 

In [25]:
print("\nTraining final model...")

# Prepare data
dtrain_final = xgb.DMatrix(X_train, label=y_train)
dtest_final = xgb.DMatrix(X_test, label=y_test)

# Train with best parameters
final_model = xgb.train(
    best_params,
    dtrain_final,
    num_boost_round=n_estimators,
    evals=[(dtrain_final, 'train'), (dtest_final, 'eval')],
    early_stopping_rounds=20,
    verbose_eval=10
)

# Make predictions and evaluate
preds_final = final_model.predict(dtest_final)
rmse_final = np.sqrt(mean_squared_error(y_test, preds_final))
mae_final = mean_absolute_error(y_test, preds_final)

print(f"\nFinal Model Performance:")
print(f"RMSE: {rmse_final:.4f}")
print(f"MAE: {mae_final:.4f}")

# Save the trained model
final_model.save_model("best_xgb_model_prem_simple.json")
print("Model saved as 'best_xgb_model_prem_simple.json'")


Training final model...
[0]	train-rmse:1.27005	eval-rmse:1.22605
[10]	train-rmse:1.16412	eval-rmse:1.17781
[20]	train-rmse:1.11454	eval-rmse:1.16433
[30]	train-rmse:1.08314	eval-rmse:1.15906
[40]	train-rmse:1.06485	eval-rmse:1.15913
[48]	train-rmse:1.05117	eval-rmse:1.15848

Final Model Performance:
RMSE: 1.1588
MAE: 0.9112
Model saved as 'best_xgb_model_prem_simple.json'


In [26]:
print("\nTop 10 Feature Importance (Gain):")
importance = final_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'importance': list(importance.values())
}).sort_values('importance', ascending=False)

print(importance_df.head(15).to_string(index=False))


Top 10 Feature Importance (Gain):
                                          feature  importance
                         team_rolling_keeper_psxg   42.617271
                                         team_elo   38.999664
     team_rolling_possession_touches_att_pen_area   38.264404
                                     opp_team_elo   20.478266
                                          is_home   15.222718
          opp_team_rolling_conceded_summary_shots   13.662328
                                     is_promoted?   13.036075
                 opp_team_rolling_summary_touches   12.119284
             opp_team_rolling_conceded_summary_xg   11.796850
                     team_rolling_summary_touches   11.783066
                   opp_team_rolling_summary_shots   11.532624
           opp_team_rolling_conceded_summary_npxg   11.461044
opp_team_rolling_conceded_summary_shots_on_target   10.932385
                team_rolling_conceded_keeper_psxg   10.926979
                   opp_team_rolling